In [5]:
import json
import os
from pathlib import Path
import shutil

src_data_set_path = Path('D:/tmp/录屏截图')

dst_data_set_path = Path('D:/.dataset/overwatch')
dst_images_path = dst_data_set_path / 'images' / 'train3'
dst_labels_path = dst_data_set_path / 'labels' / 'train3'

os.makedirs(dst_images_path)
os.makedirs(dst_labels_path)

In [6]:
def XAnyLabelimg_to_label(json_file_path: Path) -> list[tuple]:
    """
    根据X-AnyLabelimg生成的json文件，生成yolo需要的数据格式 [(类别名, 中心x, 中心y, 宽度, 高度)]
    """
    with open(json_file_path) as f:
        json_data = json.load(f)
    
    img_height = json_data['imageHeight']
    img_width = json_data['imageWidth']
    
    result = []
    for shape in json_data['shapes']:
        label = shape['label']
        
        # lt-------rt
        #  |       |
        # lb-------rb
        lt, rt, rb, lb = shape['points']
        
        center_x = (lt[0] + rt[0]) / 2 / img_width
        center_y = (lt[1] + lb[1]) / 2 / img_height
        w = (rt[0] - lt[0]) / img_width
        h = (lb[1] - lt[1]) / img_height
        
        result.append((label, center_x, center_y, w, h))
    
    return result
        
# XAnyLabelimg_to_label(Path(r'D:\.dataset\ovwatch\录制_2024_05_08_17_01_39_898.mp4_000243.367.json'))

In [7]:
def create_labels_file():
    """
    根据源文件夹中的所有 json 文件的信息，创建 label 文件
    :return: 
    """    
    # 取出源文件夹中的所有json
    all_file = os.listdir(src_data_set_path)
    all_json_file = [i for i in all_file if i.endswith('json')]
    
    for i in all_json_file:
        json_file_name = i.replace('.json', '.txt')
        json_file_path = src_data_set_path / i
        
        result = XAnyLabelimg_to_label(json_file_path)
        
        txt = '\n'.join([' '.join(map(str, i)) for i in result])
        
        with open(dst_labels_path / json_file_name, 'w') as f:
            f.write(txt)
        
create_labels_file()

In [8]:
def copy_img_by_label():
    """
    根据上一步生成的label文件名去源文件夹中取出对应的图片复制到目标文件夹
    """
    for i in os.listdir(dst_labels_path):
        label_file_name = i.replace('.txt', '')
        shutil.copy(src_data_set_path / f'{label_file_name}.jpg', dst_images_path / f'{label_file_name}.jpg')

copy_img_by_label()